In [1]:
from dash import Dash, html
import pandas as pd
import plotly.express as px
import glob

In [2]:
df_weight = pd.read_csv("../dataset/layer3/pearson_distance.csv.zip", compression="zip")

In [3]:
new_mapping = {}
count = 0

tmp = []
edge_files = glob.glob("../dataset/community/*.edge")

In [4]:
for file in edge_files:
    f = open(file, "r")
    for line in f.readlines():
        a, b = map(int, line[:-1].split())
        if a not in new_mapping:
            new_mapping[a] = count
            count += 1
        if b not in new_mapping:
            new_mapping[b] = count
            count += 1
        tmp.append([new_mapping[a], new_mapping[b], df_weight.loc[a].at[str(b)]])
        tmp.append([new_mapping[b], new_mapping[a], df_weight.loc[a].at[str(b)]])
    f.close()
    # print(file)

In [5]:
df = pd.read_csv("../dataset/new_mapping.csv")
# df.to_csv("../dataset/new_mapping.csv", index=False)
# print("fig")
fig = px.scatter(df, x="node1", y="node2", color='weight')
fig.show()

In [6]:
author = pd.read_csv("../dataset/authorToInt.csv")
id1 = author["id"].to_list()
new_col = []
# others = []
for i in id1:
    if i in new_mapping:
        new_col.append(new_mapping[i])
    else:
        new_col.append(count)
        count += 1

In [7]:
author["new_id"] = new_col

In [8]:
author.to_csv("../dataset/authorToInt.csv", index = False)

In [15]:
print(new_mapping[7014])

3474


In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

df = pd.read_csv('../dataset/combined.csv')

In [4]:
dff = df[df.community=='1']

# go.Scatter(x=dff['node1'].to_list(), y=dff['node2'].to_list(), color='topic_similarity', colorscale='Virdis')
# fig = make_subplots(rows=1, cols=2, column_widths=[plot_size, plot_size])
# fig.add_trace(row=1, col=1, trace=go.Scatter(x=dff['node1'].to_list(), y=dff['node2'].to_list(), color='topic_similarity', colorscale='Virdis')
# )

# fig.add_trace(row=1, col=2, trace=go.Scatter(x=dff['node1'].to_list(), y=dff['node2'].to_list(), color='temporal_similarity', colorscale='Plasma')
# )

# fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
# return fig

In [7]:
plot = go.Figure(data=[go.Scatter(
    x = dff['node1'].to_list(),
	y = dff['node2'].to_list(),
    mode = 'markers',
    marker=dict(
        color=dff['temporal_similarity'].to_list(),
        colorscale='Viridis',
        showscale=True
    )
)
])
                  
plot.show()